# TensorBoard example

We can integrate a server into Tensorboard so that we can see the development of our training sessions in real time as they are executing. In order to do that (see [Tensorboard documentation](https://ugr-sail.github.io/sinergym/compilation/html/pages/deep-reinforcement-learning.html#drl-logger))

We are going to use callbacks in order to do that.

In [1]:
import sinergym
from sinergym.utils.callbacks import LoggerEvalCallback, LoggerCallback
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import LoggerWrapper
from datetime import datetime
import gym
from stable_baselines3.common.logger import configure
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.vec_env import DummyVecEnv

/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


You have to define a model like DRL notebook, this model will learn with the algorithm specified in the future.

In [2]:
environment = "Eplus-demo-v1"
episodes = 4
experiment_date = datetime.today().strftime('%Y-%m-%d %H:%M')

# register run name
name = F"DQN-{environment}-episodes_{episodes}({experiment_date})"

env = gym.make(environment, reward=LinearReward)
env = LoggerWrapper(env)

[2022-08-24 09:21:36,475] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-08-24 09:21:36,477] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-08-24 09:21:36,479] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-08-24 09:21:36,482] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...


Now, we define tensorboard output path in the model definition.

In [3]:
tensorboard_path='./tensorboard_log'
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=tensorboard_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Continue with the configuration as DRL notebook.

In [4]:
n_timesteps_episode = env.simulator._eplus_one_epi_len / \
                      env.simulator._eplus_run_stepsize
timesteps = episodes * n_timesteps_episode
env_vec = DummyVecEnv([lambda: env])

We define the `LoggerCallback` in order to specify how to store the real time training data.

In [5]:
callbacks = []

# Set up Evaluation and saving best model
log_callback = LoggerCallback(True)
callbacks.append(log_callback)
# lets change default dir for TensorboardFormatLogger only
tb_path = tensorboard_path + '/' + name
new_logger = configure(tb_path, ["tensorboard"])
model.set_logger(new_logger)

callback = CallbackList(callbacks)

2022-08-24 09:21:37.692349: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-24 09:21:37.734692: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-24 09:21:37.734718: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


We have to include this callback in learning process.

In [6]:
model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=1)
model.save(env.simulator._env_working_dir_parent + '/' + name)
env.close()

[2022-08-24 09:21:39,780] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-08-24 09:21:39,788] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res6/Eplus-env-sub_run1
[2022-08-24 09:21:54,888] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2022-08-24 09:21:54,888] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-08-24 09:21:54,896] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-demo-v1-res6/Eplus-env-sub_run2
[2022-08-24 09:22:19,340] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2022-08-24 09:22:19,341] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-08-24 09:22:19,348] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /work

While training is executing, you can see that tensorboard path specified folder has been created. Then, you can start a local server and see the information about training in real-time.